In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('miss').getOrCreate()

In [2]:
df = spark.read.csv('/FileStore/tables/ContainsNull-1.csv', inferSchema=True, header = True) 

In [3]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [4]:
# row should atleast have two threshold values for dropping the values
df.na.drop(thresh = 2).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [5]:
# drops the rows if any null values
df.na.drop(how = 'any').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [6]:
# drops the rows if all values are null 
df.na.drop(how = 'all').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [7]:
df.na.drop(subset = ['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [8]:
df.printSchema()

root
-- Id: string (nullable = true)
-- Name: string (nullable = true)
-- Sales: double (nullable = true)

In [9]:
# fills string value to any string column that has missing value
df.na.fill('FILL VALUE').show()

+----+----------+-----+
 Id| Name|Sales|
+----+----------+-----+
emp1| John| null|
emp2|FILL VALUE| null|
emp3|FILL VALUE|345.0|
emp4| Cindy|456.0|
+----+----------+-----+

In [10]:
# fills up numeric value to the numeric column 
df.na.fill(0).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [11]:
df.na.fill('No Name', subset = ['Name']).show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|No Name| null|
emp3|No Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [12]:
from pyspark.sql.functions import mean

In [13]:
mean_val = df.select(mean(df['Sales'])).collect()

In [14]:
mean_sales = mean_val[0][0]

In [15]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [16]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+